In [ ]:
import torch
import torch.nn as nn
from torchvision import models
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision import datasets
from torchinfo import summary
from pathlib import Path
from sklearn.model_selection import train_test_split
from scripts import data_acquisition, data_setup, engine, plot_loss_curves, utils, model_acquisition

In [ ]:
BATCH_SIZE = 64
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
efficientnet_weights = models.EfficientNet_B2_Weights.DEFAULT
efficientnet_transforms = efficientnet_weights.transforms()
efficientnet_model = models.efficientnet_b2(weights=efficientnet_weights)

In [ ]:
resnet18_weights = models.ResNet18_Weights.DEFAULT
resnet18_transforms = resnet18_weights.transforms()
resnet18_model = models.resnet18(weights=resnet18_weights)

In [ ]:
resnet152_weights = models.ResNet152_Weights.DEFAULT
resnet152_transforms = resnet152_weights.transforms()
resnet152_model = models.resnet152(weights=resnet152_weights)

In [ ]:
vit_weights = models.ViT_B_16_Weights.DEFAULT
vit_transforms = vit_weights.transforms()
vitb16_model = models.vit_b_16(weights=vit_weights)

In [ ]:
# Run once to download the dataset
traindataset = datasets.Food101(root = 'data', download = True, transform = None, split='train')
testdataset = datasets.Food101(root = 'data', download = True, transform = None, split='test')
print(len(traindataset), len(testdataset))

In [ ]:
for param in efficientnet_model.parameters():
    param.requires_grad = False
# print(summary(efficientnet_model, input_size=(32, 3, 224, 224),col_names=("input_size", "output_size", "num_params", "trainable")))
efficientnet_model.classifier = nn.Sequential(
    nn.Dropout(p=0.2, inplace=True),
    nn.Linear(in_features=1408, out_features=101, bias=True)
)
# print(summary(efficientnet_model, input_size=(32, 3, 224, 224),col_names=("input_size", "output_size", "num_params", "trainable")))

In [ ]:
for param in resnet18_model.parameters():
    param.requires_grad = False

# print(summary(resnet18_model, input_size=(32, 3, 224, 224),col_names=("input_size", "output_size", "num_params")))
resnet18_model.fc = nn.Linear(in_features=512, out_features=101, bias=True)
# print(summary(resnet18_model, input_size=(32, 3, 224, 224),col_names=("input_size", "output_size", "num_params", "trainable")))

In [ ]:
for param in resnet152_model.parameters():
    param.requires_grad = False

# print(summary(resnet152_model, input_size=(32, 3, 224, 224),col_names=("input_size", "output_size", "num_params")))
resnet152_model.fc =  nn.Linear(in_features=2048, out_features=101, bias=True)
# print(summary(resnet152_model, input_size=(32, 3, 224, 224),col_names=("input_size", "output_size", "num_params", "trainable")))

In [ ]:
for param in vitb16_model.parameters():
    param.requires_grad = False
    
# print(summary(vitb16_model, input_size=(32, 3, 224, 224),col_names=("input_size", "output_size", "num_params")))
vitb16_model.heads.head = nn.Linear(in_features=768, out_features=101, bias=True)
# print(summary(vitb16_model, input_size=(32, 3, 224, 224),col_names=("input_size", "output_size", "num_params", "trainable")))

In [ ]:
def train_step(model:nn.Module,
               transforms:transforms.Compose,
               traindataloader:DataLoader,
               testdataloader:DataLoader,
               loss_fn:nn.Module,
               optimizer:torch.optim.Optimizer,
               ):
    train_loss, train_correct = 0, 0
    total_samples = 0
    test_loss, test_correct = 0, 0 
    model.train()
    for i, (x, y) in enumerate(traindataloader):
        x, y = x.to(device), y.to(device)
        total_samples += y.size(0)
        y_pred = model(x)
        loss = loss_fn(y_pred, y)
        train_loss += loss.item()
        batch_correct = torch.sum(torch.argmax(y_pred, dim=1) == y).item()
        train_correct += batch_correct
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if(i%100==0):
            print(f"Batch {i} : Loss {loss.item()} : Accuracy {batch_correct/y.size(0)}")
    train_accuracy  = train_correct/len(traindataloader)
    train_loss /= len(traindataloader)
    model.eval()
    total_samples = 0
    with torch.inference_mode():
        for x, y in testdataloader:
            x, y = x.to(device), y.to(device)
            total_samples += y.size(0)
            y_pred = model(x)
            loss = loss_fn(y_pred, y)
            test_loss += loss.item()
            batch_correct = torch.sum(torch.argmax(y_pred, dim=1) == y).item()
            test_correct += batch_correct
        test_accuracy = test_correct/total_samples
        test_loss /= len(testdataloader) 
    return {"train loss" : train_loss,
            "train accuracy" : train_accuracy,
            "test loss" : test_loss,
            "test accuracy" : test_accuracy
            } 

In [ ]:
def train_model(model:nn.Module,
        transforms:transforms.Compose,
        traindataloader:DataLoader,
        testdataloader:DataLoader,
        loss_fn:nn.Module,
        epochs:int,
        optimizer:torch.optim.Optimizer,):
    model.to(device)
    for epoch in range(epochs):
        print(f"Epoch {epoch}")
        results = train_step(model, transforms, traindataloader, testdataloader, loss_fn, optimizer)
        print(f"Train Loss: {results['train loss']:.4f}, Train Accuracy: {results['train accuracy']:.4f}, Test Loss: {results['test loss']:.4f}, Test Accuracy: {results['test accuracy']:.4f}")

In [ ]:
traindataset = datasets.Food101(root = 'data', download = True, split='train', transform = efficientnet_transforms)
testdataset = datasets.Food101(root = 'data', download = True, split='test', transform = efficientnet_transforms)
print(efficientnet_transforms)

In [ ]:
traindataloader = DataLoader(dataset = traindataset,
                            batch_size = BATCH_SIZE,
                            shuffle = True,)

testdataloader = DataLoader(dataset = testdataset,
                            batch_size = BATCH_SIZE,
                            shuffle = False)

In [ ]:
train_model(model = efficientnet_model,
        transforms = efficientnet_transforms,
        traindataloader = traindataloader,
        testdataloader = testdataloader,
        loss_fn = nn.CrossEntropyLoss(),
        epochs = 10,
        optimizer = torch.optim.Adam(efficientnet_model.parameters())
        )

In [ ]:
history, all_preds, all_labels, writer = engine.train(model = efficientnet_model.to(device),
                                                    model_name = "efficientnet",
                                                    data_name="food101",
                                                    epochs = 10,
                                                    train_dataloader=traindataloader,
                                                    val_dataloader = testdataloader,
                                                    device = device,
                                                    loss_fn = nn.CrossEntropyLoss(),
                                                    optimizer = torch.optim.Adam(efficientnet_model.parameters())
                                                    )

In [ ]:
torch.save(efficientnet_model.state_dict(), 'efficientnet_model_foodvision.pth')